# Iteratoren, Generatoren und Lazy Evaluation

In Python sind einige Datentypen **iterierbar**, was bedeutet, dass sie durchlaufen werden können, zum Beispiel in einer For-Schleife. Zu diesen iterierbaren Objekten gehören Listen, Tupel, Strings und auch spezielle Objekte wie `range`. Auf den ersten Blick verhalten sich eine Liste und ein `range`-Objekt sehr ähnlich, aber es gibt einen wichtigen Unterschied:

- **Liste**: Alle Elemente liegen durchgehend abrufbar im Speicher.
- **`range`-Objekt**: Die zu übergebenden Elemente werden nicht "am Stück" gespeichert, sondern erst bei Abruf jeweils einzeln erzeugt.

Dieser Unterschied hat Auswirkungen auf den Speicherbedarf und die Effizienz von Programmen, besonders bei der Arbeit mit großen Datenmengen. In dieser Lektion werden wir uns mit den Konzepten von **Iteratoren**, **Generatoren** und **Lazy Evaluation** beschäftigen und lernen, wie sie uns dabei helfen können, effizientere Programme zu schreiben.

## Listen und `range`-Objekte im Vergleich

Beginnen wir mit einem einfachen Beispiel, um den Unterschied zwischen Listen und `range`-Objekten zu verstehen.

In [1]:
# Funktion, um den Speicherplatzbedarf einer Variable zu ermitteln
from sys import getsizeof

# Erstellen einer Liste und eines range-Objekts
l = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
r = range(10)

# Ausgabe der Typen und Inhalte
print("Typ von liste:", type(l))
print("Typ von range:", type(r))
print("Inhalt von liste:", l)
print("Inhalt von range:", r)

Typ von liste: <class 'list'>
Typ von range: <class 'range'>
Inhalt von liste: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Inhalt von range: range(0, 10)


Wir sehen, dass `l` eine Listen-Objekt ist, die die Zahlen von 0 bis 9 **enthält**, während `r` ein range-Objekt ist, das die Zahlen von 0 bis 9 lediglich **repräsentiert**. Beim Ausgeben des range-Objekts sehen wir daher nur die Repräsentation `range(0, 10)` und nicht die einzelnen Zahlen.

Obwohl beide Objekte ähnlich aussehen und verwendet werden können, unterscheiden sie sich in der Art, wie sie die Daten speichern.

In [2]:
# Iteration über die Liste
print("Iteration über die Liste l:")
for e in l:
    print(e, end=" ")
print("\n")

# Iteration über das range-Objekt
print("Iteration über das range-Objekt r:")
for e in r:
    print(e, end=" ")
print("\n")

Iteration über die Liste l:
0 1 2 3 4 5 6 7 8 9 

Iteration über das range-Objekt r:
0 1 2 3 4 5 6 7 8 9 



Beide Objekte können in einer For-Schleife durchlaufen werden, und die Ausgabe ist identisch.

In [3]:
# Speicherbedarf vergleichen
print("Größe der Liste liste:", getsizeof(l), "Bytes")
print("Größe des range-Objekts range:", getsizeof(r), "Bytes")

Größe der Liste liste: 136 Bytes
Größe des range-Objekts range: 48 Bytes


Die Liste `l` benötigt mehr Speicher als das range-Objekt `r`. Dies liegt daran, dass die Liste alle Elemente im Speicher hält, während das range-Objekt nur Start-, Endwert und Schrittweite speichern muss.

Schauen wir uns an, wie sich der Speicherbedarf ändert, wenn wir die Länge der iterierbaren Objekte erhöhen.

In [4]:
large_range = range(1000000)
large_list = list(large_range)

print("Größe des range-Objekts mit 1.000.000 Elementen:", getsizeof(large_range), "Bytes")
print("Größe der Liste mit 1.000.000 Elementen:", getsizeof(large_list), "Bytes")

Größe des range-Objekts mit 1.000.000 Elementen: 48 Bytes
Größe der Liste mit 1.000.000 Elementen: 8000056 Bytes


**Erläuterung:**

- Der Speicherbedarf des `range`-Objekts bleibt praktisch konstant, selbst bei einer Million Elemente.
- Die Liste ist dagegen ungefähr 8 MB groß!

**Hinweis:**

Die Funktion `getsizeof` misst nur die Größe des Objekts selbst und nicht den Speicherbedarf der einzelnen Elemente - dieser kann je nach Speicherverwaltung auch höher liegen. Dennoch verdeutlicht dies den Unterschied zwischen den beiden Objekttypen.

## Iterablen und Iteratoren

- **Iterablen**: Eine Iterable ist ein Objekt, das eine `__iter__()`-Methode implementiert, die einen Iterator zurückgibt. Iterablen sind Objekte, über die man in einer Schleife iterieren kann. Beispiele für eingebaute Iterable-Objekte in Python sind Listen, Tupel, Strings und Mengen.

- **Iteratoren**: Ein Iterator ist ein Objekt, das die Methoden `__iter__()` und `__next__()` implementiert. Die Methode `__next__()` gibt das nächste Element des Iterators zurück und wirft eine `StopIteration`-Exception, wenn keine weiteren Elemente vorhanden sind.

**Hinweis:** Dunder-Methoden (Double-underscore-Methoden) wie `__next__()` oder `__iter__()` werden in Python meist nicht direkt aufgerufen. Stattdessen greift man auf diese spezielle Funktionalität durch dafür vorgesehene Hilfsfunktionen oder Sprachkonstrukte zu. Beispielsweise verwendet man die eingebaute `next()`-Funktion, um das nächste Element von einem Iterator zu erhalten, anstatt direkt `iterator.__next__()` aufzurufen. Dies fördert klareren und lesbareren Code und nutzt die volle Flexibilität, die Python bietet.

Bei der Iteration über eine Iterable in einer Schleife wird implizit ein Iterator erstellt, um die Iteration durchzuführen. Hier ist ein einfaches Beispiel:

In [5]:
# Einfache Liste als Iterable
liste = [1, 2, 3]

# Erstellen eines Iterators aus der Iterablen
iterator = iter(liste)
# Alternativ: iterator = liste.__iter__()

# Abrufen von Elementen mit einem Iterator
print(next(iterator))  # Gibt 1 zurück
print(next(iterator))  # Gibt 2 zurück
print(next(iterator))  # Gibt 3 zurück
#print(next(iterator))  # Wirft eine StopIteration Exeption

1
2
3


## Funktionen, die Iteratoren zurückgeben

Einige eingebaute Funktionen in Python geben Iteratoren zurück. Beispiele sind `map`, `zip` und `enumerate`.

#### `map`

In [6]:
l1 = [1, 2, 3, 4, 5]

# Verwenden von map
m = map(lambda x: x**2, l1)

print("Typ von m:", type(m))
print(next(m)) # Die erste Zahl außerhalb der for-Schleife

for item in m:
    print(item, end=" ")
print("\n")

Typ von m: <class 'map'>
1
4 9 16 25 



#### `zip`

In [7]:
# Erzeugen wir zunächst eine zweite Liste l2, mit der wir l1 "zippen" können
l2 = [1, 4, 9, 16, 25]

# "Automatische" Erzeugung der Liste mit map
# l2 = list(map(lambda x: x**2, l1))

# "Automatische" Erzeugung der Liste mit List Comprehension
# l2 = [x**2 for x in l1]

print(l2)

[1, 4, 9, 16, 25]


In [8]:
# Verwenden von zip
z = zip(l1, l2)

print("Typ von z:", type(z))
print(next(z))  # Gibt das erste Tupel zurück

for item in z:
    print(item, end=" ")
print("\n")

Typ von z: <class 'zip'>
(1, 1)
(2, 4) (3, 9) (4, 16) (5, 25) 



#### `enumerate`

In [9]:
# Verwenden von enumerate
e = enumerate(l1)

print("Typ von e:", type(e))
print(next(e))  # Gibt das erste Tupel (Index, Wert) zurück

for index, value in e:
    print(f"Index {index}: Wert {value}")

Typ von e: <class 'enumerate'>
(0, 1)
Index 1: Wert 2
Index 2: Wert 3
Index 3: Wert 4
Index 4: Wert 5


Die Iteratoren generieren ihre Elemente erst bei der Iteration. Dies spart Speicher, da nicht alle Elemente im Voraus berechnet und gespeichert werden müssen.<br>
<br>
## ⚠️ 
#### `range`
Anders als man zunächst vielleicht vermuten könnte, ist das Objekt, welches die range-Funktion zurückgibt, an und für sich erst einmal **kein** Iterator, sondern eine Iterable, da es über keine eigene `__next__`-Methode verfügt. 

In [56]:
print(type(range(n)))

<class 'range'>


Der entsprechende Iterator muss erst noch mit der `__iter__()`-Methode des range-Objekts erzeugt werden. In der Regel passiert das, ohne dass wir uns selbst darum kümmern müssen - z.B. durch eine `for`-Schleife, welche die `__iter__`-Methode des range-Objekts aufruft und den zurückgegebenen Iterator "auffängt" und verwaltet, und dabei auch die `next`-Aufrufe übernimmt. Die for-Schleife nimmt uns also eine Menge Arbeit ab, und sie tut das komplett im Hintergrund.<br>
<br>
Es geht aber auch manuell:

In [66]:
r = iter(range(6)) 
# iter() ist eine Funktion, welche hier die __iter__-Methode des range-Objekts aufruft

print("Typ von r:", type(r))

Typ von r: <class 'range_iterator'>


In [67]:
# Manuelles Abrufen von Elementen mit dem Iterator
print(next(r))
print(next(r))
print(next(r))

# Was passiert, wenn wir nun diesen bereits "halb verbrauchten" Iterator einer Schleife übergeben?
print("Fortgesetzte Iteration über r:", end=" ")
for item in r:
    print(item, end=" ")

0
1
2
Fortgesetzte Iteration über r: 3 4 5 

## Eigene Iteratoren mit Generatoren erstellen

Wenn wir Funktionen schreiben, die große Datenmengen verarbeiten, ist es hilfreich, wenn sie Iteratoren zurückgeben. Dies können wir mit Generatoren erreichen.

### Generator-Funktionen mit `yield`

Durch das Schlüsselwort `yield` können wir Funktionen schreiben, die bei jedem Aufruf von `next()` ein neues Element liefern.

In [11]:
def simple_generator():
    print("Ausgabe beim ersten Aufruf:", end=" ")
    yield 1
    print("Ausgabe beim zweiten Aufruf:", end=" ")
    yield 2
    print("Ausgabe beim dritten Aufruf:", end=" ")
    yield 3

gen = simple_generator()

print("Typ von simple_generator:", type(simple_generator))
print("Typ von simple_generator():", type(simple_generator()))
print("Typ von gen:", type(gen))

Typ von simple_generator: <class 'function'>
Typ von simple_generator(): <class 'generator'>
Typ von gen: <class 'generator'>


In [12]:
# Führe diese Zelle mehrmals aus, um die Ausgaben zu sehen
print(next(gen))

Ausgabe beim ersten Aufruf: 1


Was passiert hier?<br>
Die Funktion `simple_generator()` liefert einen Generator, den wir der Variablen `gen` zuweisen. Durch den Aufruf von `next(gen)` wird der Funktionskörper zunächst bis zum ersten `yield` ausgeführt. Außerdem wird
- der Ausdruck hinter diesem `yield` zurückgegeben (ähnlich wie bei `return`), 
- der aktuelle Zustand des Generators zwischengespeichert,
- die weitere Ausführung des Generators angehalten und 
- erst beim nächsten Aufruf von `next(gen)` fortgesetzt - bis zum nächsten `yield` und so weiter,
bis `gen` schließlich "leer läuft".

Anstatt den Generator manuell in einer Variablen zu speichern, können wir ihn auch direkt verwenden, zum Beispiel in einer for-Schleife. Die "Verwaltung" des Generators inklusive dem Aufrufen den jeweils nächsten Elements übernimmt dann die Schleife im Hintergrund.

In [13]:
for aufruf in simple_generator():
    print(aufruf)

Ausgabe beim ersten Aufruf: 1
Ausgabe beim zweiten Aufruf: 2
Ausgabe beim dritten Aufruf: 3


⚠️ Beim direkten Aufruf wird jedes mal eine neue Instanz des Generators erzeugt, die auch jedes mal wieder mit der ersten `yield`-Ausgabe beginnt.

In [14]:
print(next(simple_generator()))
print(next(simple_generator()))
print(next(simple_generator()))


Ausgabe beim ersten Aufruf: 1
Ausgabe beim ersten Aufruf: 1
Ausgabe beim ersten Aufruf: 1


Nach dem Ausführen dieser Codezelle liegen irgendwo im Arbeitsspeicher drei Instanzen des Generators herum, die alle auf ihren jeweils zweiten Abruf warten - der nie kommen wird, weil sie ohne die Zuweisung zu einer Variablen wie der hier oft verwendeten `gen` keinen Namen haben und daher gar nicht mehr mehr aufgerufen werden können.
Der direkte Aufruf ist daher nur in Kontexten wie for-Schleifen sinnvoll, wo die erzeugte Generator-Instanz im Hintergrund "aufgefangen" und verwaltet wird. Außerhalb solcher Kontexte gelangen wir ohne die Zuweisung zu einer Variablen wie hier `gen` immer nur höchstens bis zur ersten `yield`-Ausgabe, während wir den Iterator als unbrauchbaren Datenmüll im Speicher hinterlassen.<br>
#### 💡
In der Praxis ist es meist sinnvoller, nicht jeden `yield` einzeln ausdrücklich und manuell zu auszuformulieren - sonst könnte man die entsprechenden Ausgaben auch einfach in eine Liste schreiben und diese durchgehen. Stattdessen bietet sich ein dynamischer Aufruf an, z.B. durch eine Schleife:

In [15]:
# Geben wir dem Generator auch gleich noch einen Parameter mit
def my_generator(n): # Ein Generator, der nach und nach die Zahlen von 1 bis n erzeugt
    i = 1
    while i <= n:
        yield i
        i += 1

gen = my_generator(10)

In [16]:
# Ziehen wir die ersten beiden Ausgaben des Generators manuell:
print("Erste Zahl aus dem Generator:", next(gen))  # Gibt 1 zurück
print("Zweite Zahl aus dem Generator:", next(gen))  # Gibt 2 zurück

# Iterieren über "den Rest" von gen:
for num in gen:
    print(num)

Erste Zahl aus dem Generator: 1
Zweite Zahl aus dem Generator: 2
3
4
5
6
7
8
9
10


💥 Der Ausdruck hinter dem `yield` ist nicht auf Zahlen beschränkt, sondern kann ein Ausdruck jeglicher Art sein, wie eine Variable, eine Funktion oder die Prüfung auf einen Wahrheitswert. Wird gar nichts hinter yield geschrieben, wir `None` zurückgegeben.

In [ ]:
# Ein Iterator, der bei jedem zweiten Aufruf ein Tupel zurückgibt (Anzahl der Aufrufe, Wahrheitswert von "ist durch 3 teilbar"),
# bei den anderen ("ungeradzahligen") Aufrufen dagegen ein None.
def my_generator(n):
    i = 1
    while i <= n:
        if i % 2 == 0:
            yield (i, i % 3 == 0)
        else:
            yield 
        i += 1

for num in my_generator(12):
    print(num)

None
(2, False)
None
(4, False)
None
(6, True)
None
(8, False)
None
(10, False)
None
(12, True)


### Vergleich von Funktionen mit und ohne Generatoren

**Zwei Wege, die Quadrate aller Zahlen bis `n` zu erzeugen**

In [ ]:
# Funktion, die eine Liste zurückgibt
def squares_list(n):
    result = []
    for i in range(n):
        result.append(i**2)
    return result

# Funktion, die einen Generator zurückgibt
def squares_generator(n):
    for i in range(n):
        yield i**2

**Ausgabe und Speicherbedarf im Vergleich**

In [45]:
# Erzeugen wir beide Objekte
n = 500
squares = squares_list(n)
squares_gen = squares_generator(n)

# Quadratzahlen als Liste
print("Quadratzahlen als Liste:", squares)
print("Größe der Liste:", getsizeof(squares), "Bytes")
print()

# Quadratzahlen als Generator
print("Quadratzahlen als Generator:", squares_gen)
print("Quadratzahlen aus Generator:", end=" ")
for num in squares_gen: 
    print(num, end=" ")
print("\nGröße des Generators:", getsizeof(squares_gen), "Bytes")

Quadratzahlen als Liste: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801, 10000, 10201, 10404, 10609, 10816, 11025, 11236, 11449, 11664, 11881, 12100, 12321, 12544, 12769, 12996, 13225, 13456, 13689, 13924, 14161, 14400, 14641, 14884, 15129, 15376, 15625, 15876, 16129, 16384, 16641, 16900, 17161, 17424, 17689, 17956, 18225, 18496, 18769, 19044, 19321, 19600, 19881, 20164, 20449, 20736, 21025, 21316, 21609, 21904, 22201, 22500, 22801, 23104, 23409, 23716, 24025, 24336, 24649, 24964, 25281, 

**Erläuterung:**

- Die Listenfunktion `squares_list` erstellt eine Liste aller Quadratzahlen und speichert sie. Effizient bei wenigen Elementen.
- Die Generatorfunktion `squares_generator` liefert die Quadratzahlen sequenziell auf Abruf, benötigt einen konstanten Speicherplatz (unabhängig von der Zahl der zu erzeugenden Elemente) und ist damit deutlich speichereffizienter, wenn eine größere Anzahl Elemente benötigt wird.

### Optional: enumerate, zip und map - aber selbst gebaut

#### `enumerate`

In [ ]:
def my_enumerate(iterable, start=0):    # Nimm eine Iterable und einen Startwert mit dem Standardwert 0
    index = start
    it = iter(iterable)                 # Erzeuge einen Iterator aus der Iterable
    while True:                         # Beginne eine Endlosschleife
        try:                            # Versuche, den folgenden Code auszuführen
            yield (index, next(it))     # Gib ein Tupel (Index, nächstes Element) zurück
        except StopIteration:           # Fange die StopIteration Ausnahme ab
            return                      # Beende die Funktion (und damit die Schleife)
        index += 1                      # Erhöhe den Index um 1

print(type(my_enumerate))
print(type(my_enumerate([])))

<class 'function'>
<class 'generator'>


In [ ]:
# Beispiel für `my_enumerate`
colors = ['red', 'green', 'blue']

enumerated_colors = my_enumerate(colors, start=1)

for index, color in enumerated_colors:
    print(index, color)

1 red
2 green
3 blue


#### `zip`

In [68]:
def my_zip(*iterables): # Beachte: Durch das * können beliebig viele Iterables als Argumente übergeben werden
                        # Vgl. *args in der Live Session zu Funktionen
    iterators = [iter(i) for i in iterables]        # Erzeuge eine Liste von Iteratoren
    while True:
        try:
            result = [next(i) for i in iterators]   # Rufe next() für jeden Iterator auf und sammle die Ergebnisse
            yield tuple(result)                     # Gib die Ergebnisse als Tupel zurück
        except StopIteration:
            return

In [70]:
# Beispiel für `my_zip`
numbers = [1, 2, 3]
letters = ['a', 'b', 'c']
booleans = [True, False, True]

for number, letter, boolean in my_zip(numbers, letters, booleans):
    print(f"number: {number}, letter: {letter}, boolean: {boolean}")

number: 1, letter: a, boolean: True
number: 2, letter: b, boolean: False
number: 3, letter: c, boolean: True


#### `map`

In [71]:
def my_map(f, *iterables):          # Nimm eine Funktion f und beliebig viele Iterables als Argumente
    for args in my_zip(*iterables): # Verwende my_zip, um die Iterables parallel zu durchlaufen
        yield f(*args)              # Wende die Funktion f auf die aktuellen Argumente an und gib das Ergebnis zurück

In [ ]:
# Beispiel für `my_map`: "Spaltenweises" aufsummieren beliebig vieler beliebig langer Zahlenlisten

list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
list2 = [3, 1, 8, 9, 5, 7, 2, 6, 4]
list3 = [6, 5, 4, 3, 2, 1, 9, 8, 7]
list4 = [5, 6, 8, 1, 9, 3, 4, 2, 7]

zipped = my_zip(list1, list2, list3, list4)

# Summiere die Elemente der drei Listen "spaltenweise"
for summed_column in my_map(sum, zipped):
    print(summed_column)

15
14
23
17
21
17
22
24
27


### Generator Expressions

Ähnlich zu List Comprehensions können wir Generator Expressions verwenden, um Generatoren zu erstellen.

In [ ]:
# List Comprehension
lc = [i**2 for i in range(100)]
print("List Comprehension:", lc)
print("Größe von lc:", getsizeof(lc), "Bytes")

# Generator Expression
ge = (i**2 for i in range(100))
print("Generator Expression:", ge)
print("Größe von ge:", getsizeof(ge), "Bytes")

print("Quadratzahlen aus Generator Expression:")
for num in ge:
    print(num, end=" ")
print()

**Erläuterung:**

- Die List Comprehension erstellt sofort eine Liste mit allen Elementen.
- Die Generator Expression erzeugt die Elemente bei Bedarf und verbraucht weniger Speicher.





## Einsatz von Generatoren für Lazy Evaluation

**Lazy Evaluation** bedeutet, dass Berechnungen erst dann durchgeführt werden, wenn sie tatsächlich benötigt werden. Generatoren ermöglichen es uns, dieses Konzept in Python zu nutzen.

**Beispiel mit unendlicher Folge von Fibonacci-Zahlen:**

Fibonacci-Zahlen sind eine berühmte Zahlenfolge, die aus der Summe der beiden vorherigen Zahlen gebildet wird. Die Folge beginnt mit 0 und 1, und jedes nachfolgende Element ist die Summe der vorhergehenden beiden. 

$$
F(n) = 
\begin{cases} 
0, & \text{wenn } n = 0 \\
1, & \text{wenn } n = 1 \\
F(n-1) + F(n-2), & \text{wenn } n > 1
\end{cases}
$$

Dabei steht $ F(n) $ für die $ n $-te Fibonacci-Zahl.



| $n$ | $F(n)$ |
|-----|--------|
|0    | 0      |
|1    | 1      |
|2    | 1      |
|3    | 2      |
|4    | 3      |
|5    | 5      |
|6    | 8      |
|7    |13      |

Durch den Einsatz von Lazy Evaluation mit Generatoren können wir diese unendliche Folge generieren, ohne unendlich viel Speicher zu benötigen. Wir können bei Bedarf jederzeit das nächste Element erzeugen.

In [ ]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

fib_gen = fibonacci()

# Ausgabe der ersten 20 Fibonacci-Zahlen
print(f"{'n':>3} | {'F(n)':<5}")
print("-"*11)

for n, f in zip(range(20), fib_gen):
    print(f"{n:>3} | {f:>5}")

**Erläuterung:**

- Die Funktion `fibonacci` erzeugt eine unendliche Zahlenfolge.
- Da die Zahlen bei Bedarf generiert werden, können wir theoretisch unendlich viele Elemente haben, ohne den Speicher zu überlasten.

## Nachteile von Generatoren gegenüber Listen

Generatoren sind speicher- und zeiteffizient, haben aber Nachteile gegenüber Listen:

- **Nicht indexierbar**: Kein direkter Zugriff auf Elemente per Index.
- **Einmalige Iteration**: Nach vollständiger Durchlaufung erschöpft.
- **Unbekannte Länge**: `len()` funktioniert nicht, da Länge nicht definiert.
- **Kein zufälliger Zugriff**: Keine Möglichkeit, Elemente zu überspringen oder zurückzugehen.
- **Erschwertes Debugging**: Zustand schwer einsehbar.

**Beispiele:**

In [ ]:
# Nicht indexierbar und Einmalige Iteration
try:
    my_generator = (x for x in range(5))
    print("Indexzugriff:", my_generator[2])  # Fehler
except TypeError as e:
    print("Fehler:", e)

In [ ]:
# Einmalige Iteration
gen = (x for x in range(3))
print("Erster Durchlauf:")
for value in gen:
    print(value)
print("Zweiter Durchlauf:")
for value in gen:
    print(value)  # Keine Ausgabe

In [ ]:
# Unbekannte Länge
try:
    gen = (x for x in range(5))
    print(len(gen))  # Fehler
except TypeError as e:
    print("Fehler:", e)

## Zusammenfassung

1. **Speicherbedarf von `range` und Listen**: `range`-Objekte speichern nur Start, Ende und Schrittweite und verbrauchen konstanten Speicher, während Listen alle Elemente im Speicher halten.

2. **Iteratoren**: Objekte wie `map`, `zip` und `enumerate` geben Iteratoren zurück, die ihre Elemente bei Bedarf generieren.

3. **Generatoren**: Mit dem `yield`-Schlüsselwort können wir Generatoren erstellen, die Iteratoren sind und Lazy Evaluation ermöglichen.

4. **Generator Expressions**: Ähnlich zu List Comprehensions, aber sie erzeugen Elemente bei Bedarf.

5. **Lazy Evaluation**: Berechnungen werden erst durchgeführt, wenn sie benötigt werden, was Speicher und Rechenleistung spart.

## f-strings
Vielleicht sind Dir in diesem Notebook Code-Snippets aufgefallen, die ungefähr so aussahen:
``` Python
f"text {variable} text {expression} text."
```
Hierbei handelt es sich um sogenannte **f-Strings**. f-Strings sind eine komfortable Methode, um auf gut lesbare Art und Weise Platzhalter für Variablen und andere Ausdrücke in Strings einzufügen.<br>
<br>
Sie werden erzeugt, indem dem einleitenden Anführungszeichen eines Strings ohne trennendes Leerzeichen der Buchstabe "f" vorangestellt wird. Im String selbst können nun an beliebigen Stellen beliebig viele geschweifte Klammern "{}" gesetzt werden, in welche nun Ausdrücke wie Variablen oder Funktionen hineingeschrieben werden können. Der (Rückgabe-) Wert dieser Ausdrücke wird dann an der entsprechenden Stelle im String eingefügt.<br>
<br>
f-Strings bieten noch ein paar weitere Möglichkeiten, die hier aber etwas zu weit vom Thema des Tages wegführen würden. Sie werden aber in der Python Basics Live Session "**Erweiterte Python Syntax**" ausführlicher vorgestellt. Wir verwenden sie auch in anderen Live Sessions wie dieser; allerdings stets so, dass wir uns ihre Vorteile in der Darstellung zunutze machen, ohne dass sie sich auf den thematischen Inhalt der Live Sessions auswirken.